In [1]:
import xarray as xr
import numpy as np
import cmocean as cm
import xgcm

In [28]:
import numpy as np

def mi96_1d(pH, pdzmin, ph_co, pkth, kkconst, pacr, jpk):
    """
    Calculate depth levels for a given set of input parameters using Madec & Imbard (1996) function.

    Parameters:
    - pH: Depth of the bottom [meters]
    - pdzmin: Value of e3 at the connection point [meters]
    - ph_co: Depth of the connection between z- and s-coordinates [meters]
    - pkth: Position of the inflexion point
    - kkconst: Number of levels with z-coordinates [meters]
    - pacr: Slope of the tanh function
    - jpk: Number of vertical levels
    - ldprint: Whether to print information about the arguments (default False)

    Returns:
    - zlev_dep: Depth of the levels (2D numpy array with shape (2, jpk))
    """
    
    zlev_dep = np.zeros((2, jpk))

    # Compute parameters za0, za1, and zsur
    za1 = (pdzmin - (pH - ph_co) / (jpk - 1 - kkconst)) / (
        np.tanh((1 - pkth) / pacr) -
        pacr / (jpk - 1 - kkconst) * (np.log(np.cosh((jpk - kkconst - pkth) / pacr)) -
                                  np.log(np.cosh((1 - pkth) / pacr)))
    )
    
    za0 = pdzmin - za1 * np.tanh((1 - pkth) / pacr)
    zsur = -za0 - za1 * pacr * np.log(np.cosh((1 - pkth) / pacr))

    # Calculate depth at T and W-points
    for jk in range(kkconst + 1, jpk + 1):
        zw = float(jk - kkconst)
        zt = float(jk - kkconst) + 0.5

        zlev_dep[0, jk - 1] = zsur + za0 * zw + za1 * pacr * np.log(np.cosh((zw - pkth) / pacr)) + ph_co
        zlev_dep[1, jk - 1] = zsur + za0 * zt + za1 * pacr * np.log(np.cosh((zt - pkth) / pacr)) + ph_co

    return zlev_dep

def zgr_sco_mi96( pHmax, pdzmin, pkth, ph_co, pacr, jpk):
    """
    Define full vertical s-coordinate system using the 2D bathymetry field.

    Parameters:
    - pdept_1d, pdepw_1d: 1D t- and w-depth (numpy arrays)
    - pbathy: Bathymetry (2D numpy array)
    - pHmax: Maximum depth [meters]
    - pdzmin: Minimum value of e3 at the surface [meters]
    - pkth: Position of the inflexion point
    - ph_co: Depth of the connection between z- and s-coordinates [meters]
    - pacr: Slope of the tanh function
    - pe3t_1d, pe3w_1d: 1D t- and w-points vertical scale factors (numpy arrays)
    - pdept, pdepw: 3D t and w-depth (3D numpy arrays)
    - pe3t, pe3u, pe3v, pe3f, pe3w, pe3uw, pe3vw: Vertical scale factors (3D numpy arrays)
    - jpk: Number of vertical levels
    - lwp: Whether to print information about the arguments (default False)
    """

    # 1D profile
    zlev_dep = mi96_1d(pHmax, pdzmin, 0.0, pkth, 0, pacr, jpk)
    pdepw_1d = zlev_dep[0, :]
    pdept_1d = zlev_dep[1, :]

    # Nearest index and its depth value to the depth where s-coordinates should start
    kkconst = np.argmin(np.abs(pdepw_1d - ph_co))
    
    # compute e3w from depth:
    pe3w_1d     = 2. * ( pdept_1d - pdepw_1d ) 
    pe3w_1d[1:] = pdept_1d[1:] - pdept_1d[:-1]

    zlev_dep = mi96_1d(pHmax, pe3w_1d[kkconst], pdepw_1d[kkconst], pkth, kkconst, pacr, jpk)

    return(zlev_dep)
    # # initialise depth arrays
    # pdepw = np.tile(pdepw_1d, (pbathy.shape[0], pbathy.shape[1], 1))
    # pdept = np.tile(pdept_1d, (pbathy.shape[0], pbathy.shape[1], 1))

    # # 3D profile
    # for ji in range(pbathy.shape[0]):
    #     for jj in range(pbathy.shape[1]):
    #         zlev_dep = mi96_1d(pbathy[ji, jj], pe3w_1d[kkconst], pdepw_1d[kkconst], pkth, kkconst - 1, pacr, jpk)
    #         pdepw[ji, jj, :] = pdepw_1d[:]
    #         pdept[ji, jj, :] = pdept_1d[:]

    #         for jk in range(kkconst + 1, jpk):
    #             pdepw[ji, jj, jk] = zlev_dep[0, jk]
    #             pdept[ji, jj, jk] = zlev_dep[1, jk]


In [32]:
# Example usage:
pH = 4000.0
pdzmin = 10.0
ph_co = 1000
pkth = 35
kkconst = 0
pacr = 10.5
jpk = 36

zlev_dep = zgr_sco_mi96(pH, pdzmin, pkth, ph_co, pacr, jpk)
print("zlev_dep:", zlev_dep[0])
print("zlev_dep:", zlev_dep[1])

zlev_dep: [   0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
  982.37575434 1134.97176404 1305.79569866 1498.65489118 1718.14753809
 1969.82495315 2260.38617508 2597.91094825 2992.13801384 3454.79660695
 4000.        ]
zlev_dep: [   0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
 1056.60619264 1217.88411422 1399.20379319 1604.74952324 1839.57388643
 2109.77548626 2422.71193351 2787.25455555 3214.09224332 3716.09281026
 4308.73114341]


In [30]:
OUTPUT_FILES = {
    "snapshot": "acc2.snapshot.nc",
    "averages": "acc2.averages.nc",
    "overturning": "acc2.overturning.nc",
    "energy": "acc2.energy.nc",
}

In [12]:
ds_snapshot = xr.open_dataset(OUTPUT_FILES["snapshot"])

In [13]:
ds_snapshot.temp.isel(zt=0, Time=0).plot()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
ds_snapshot.u.isel(zt=0, Time=-2).plot()

IndexError: index -2 is out of bounds for axis 0 with size 1

In [26]:
ds_snapshot

<xarray.Dataset> Size: 90MB
Dimensions:            (xt: 30, xu: 30, yt: 42, yu: 42, zt: 15, zw: 15,
                        tensor1: 2, tensor2: 2, isle: 2, Time: 36)
Coordinates:
  * xt                 (xt) float64 240B -1.0 1.0 3.0 5.0 ... 53.0 55.0 57.0
  * xu                 (xu) float64 240B 0.0 2.0 4.0 6.0 ... 52.0 54.0 56.0 58.0
  * yt                 (yt) float64 336B -41.0 -39.0 -37.0 ... 37.0 39.0 41.0
  * yu                 (yu) float64 336B -40.0 -38.0 -36.0 ... 38.0 40.0 42.0
  * zt                 (zt) float64 120B -1.942e+03 -1.666e+03 ... -26.0 -14.0
  * zw                 (zw) float64 120B -1.804e+03 -1.548e+03 ... -20.0 0.0
  * tensor1            (tensor1) float64 16B 0.0 1.0
  * tensor2            (tensor2) float64 16B 0.0 1.0
  * isle               (isle) float64 16B 0.0 1.0
  * Time               (Time) timedelta64[ns] 288B 10 days 20 days ... 360 days
Data variables: (12/37)
    dxt                (xt) float64 240B 2.224e+05 2.224e+05 ... 2.224e+05
    dxu                (xu) float64 240B 2.224e+05 2.224e+05 ... 2.224e+05
    dyt                (yt) float64 336B 2.224e+05 2.224e+05 ... 2.224e+05
    dyu                (yu) float64 336B 2.224e+05 2.224e+05 ... 2.224e+05
    dzt                (zt) float64 120B ...
    dzw                (zw) float64 120B ...
    ...                 ...
    forc_rho_surface   (Time, yt, xt) float64 363kB ...
    psi                (Time, yu, xu) float64 363kB ...
    psin               (isle, yu, xu) float64 20kB ...
    tke                (Time, zw, yt, xt) float64 5MB ...
    forc_tke_surface   (Time, yt, xt) float64 363kB ...
    eke                (Time, zw, yt, xt) float64 5MB ...
Attributes:
    date_created:       2024-03-28T16:12:14.394574
    veros_version:      0+untagged.1774.g4039f76
    setup_identifier:   acc
    setup_description:  My ACC setup
    setup_settings:     {"identifier": "acc", "description": "My ACC setup", ...
    setup_file:         /Users/davidkamm/PhD/veros_DINO/veros/setups/acc/acc.py
    setup_code:         #!/usr/bin/env python\n\nfrom veros import VerosSetup...

In [23]:
ds_snapshot.dxt.values

array([222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408, 222354.94670408, 222354.94670408,
       222354.94670408, 222354.94670408])

In [25]:
ds_snapshot.area_t.values

array([[3.73141415e+10, 3.73141415e+10, 3.73141415e+10, ...,
        3.73141415e+10, 3.73141415e+10, 3.73141415e+10],
       [3.84234348e+10, 3.84234348e+10, 3.84234348e+10, ...,
        3.84234348e+10, 3.84234348e+10, 3.84234348e+10],
       [3.94859151e+10, 3.94859151e+10, 3.94859151e+10, ...,
        3.94859151e+10, 3.94859151e+10, 3.94859151e+10],
       ...,
       [           nan,            nan, 3.94859151e+10, ...,
        3.94859151e+10, 3.94859151e+10, 3.94859151e+10],
       [           nan,            nan, 3.84234348e+10, ...,
        3.84234348e+10, 3.84234348e+10, 3.84234348e+10],
       [           nan,            nan, 3.73141415e+10, ...,
        3.73141415e+10, 3.73141415e+10, 3.73141415e+10]])